# Trading Data Specialist interview
## Code test

### Instructions
The purpose of the exercise is for you to showcase your ability to manipulate data and store it efficiently. 
You are expected to solve the task using python as the main language, ideally not using more than three hours of your time.
All code is to be sent as a zip file or a github link to EDSIM@orsted.dk before your interview.


For this case, we would like you to expose our trading data through an REST API, so it can be accessed by analysts and traders. 
The following function generates trade data. You can consider this as mocking a connection to our trading system.

In [10]:
import pandas as pd
import numpy as np

def get_raw_data(dt_from: pd.Timestamp, dt_to: pd.Timestamp) -> pd.DataFrame:
    """
    Generates (random) trade data
    :param dt_from (pd.Timestamp): datetime from which to load the data
    :param dt_to (pd.Timestamp): datetime up to which to load the data
    """

    dt_from = pd.to_datetime("now")
    dt_to = dt_from + pd.to_timedelta(7, unit="d")
    n_rows = int(1e6)
    min_price = -100
    max_price = 300
    min_vol = -100
    max_vol = 100
    cols = [
        "trade_id",  # integer string identifying trade
        "execution_time",  # time at which the trade was executed
        "price",  # price in EUR/MWh 
        "volume",  # volume in MWh (negative = buy, positive = sell)
        "market",  # country
        "product"  # name of the product
    ]
    markets = ["uk", "nl", "de"]
    products = ["XBID_2h", "XBID_1h", "XBID_30min", "local_2h", "local_1h", "local_30min"]

    df_raw = pd.DataFrame(index=range(n_rows), columns=cols)
    df_raw["product"] = np.random.choice(products, size=n_rows)
    df_raw["market"] = np.random.choice(markets, size=n_rows)
    df_raw["execution_time"] = pd.to_datetime(
        np.random.uniform(
            dt_from.to_numpy(), 
            dt_to.to_numpy(), 
            size=n_rows
        )
    ).round("1ms")
    df_raw["price"] = np.random.uniform(
        min_price, 
        max_price, 
        size=n_rows
    )
    df_raw["volume"] = np.random.uniform(
        min_vol, 
        max_vol, 
        size=n_rows
    )
    df_raw["trade_id"] = np.random.randint(n_rows, n_rows*2, size=n_rows).astype(str)
    df_raw["product_duration"] = df_raw["product"].str.split("_").str[-1]
    df_raw["product_type"] = df_raw["product"].str.split("_").str[0]

    return df_raw

The task is to create a data pipeline to dump the data in a batch fashion into a database. You should set up the database and the pipeline as you see fit, ideally as you would working in Azure (not mandatory). Make sure to document all your code to do that. The database setup should be optimized towards speed for the use cases described below.

The data should be served from a REST API. It should be possible to extract aggregations of the trade data (price and volume) across any combination of the other columns. It should be possible to perform:
- simple average
- volume weighted average (using the other column as weights)
- sum
- minimum 
- maximum
- average of n largest/smallest elements
At the same time, it should be possible to track the aggregation over time (as a function of execution time, rounded to a given frequency).
For example, the user should be able to query the development over time (e.g. every 10 minutes) of the volume weighted average price of a given type of product.


For example, API calls could look as below. The example illustrates how a user could access the development of the minimum price per market and product type, for a product duration of 30min in the month of January, at sampling intervals of 10 minutes.

In [12]:
"https://trades.api.com?col=price&agg=min&groupby=market,product_type&product_duration=30min&from=202101010000&to=202102010000&freq=10min"

'https://trades.api.com?col=price&agg=min&groupby=market,product_type&product_duration=30min&from=202101010000&to=202102010000&freq=10min'